In [1]:
#Auto relode the notebooks
%load_ext autoreload
%autoreload 2

In [2]:
#import python libraries
import os
import warnings
import sys
import datetime

import dvc.api
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.preprocessing import LabelEncoder
from fast_ml.model_development import train_valid_test_split
from sklearn.tree import DecisionTreeClassifier
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import pickle

In [5]:
#import Scripts
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from plot_data import PlotData
from preprocessing import Preprocess
from load_data import LoadData

In [6]:
loader = LoadData()

In [7]:
#get the data
train_path = 'data/train_cleaned.csv'
test_path = 'data/test_cleaned.csv'
repo = 'https://github.com/yonamg/Pharmaceutical-Sales-Prediction'
tra_ver = '48de72c'
tst_ver = '3e8c890'